In [21]:
#versão da Linguagem Python
from platform import python_version
print('Versão da linguagem Python usada no ambiente virtual desse projeto:', python_version())

Versão da linguagem Python usada no ambiente virtual desse projeto: 3.12.7


In [22]:
import pandas as pd
import numpy as np
import os
import re 
import openpyxl
import csv
import os
from IPython.display import display, HTML


#### Fonte: "IBGE - Índice Nacional de Preços ao Consumidor Amplo (IPCA)

In [23]:
import pandas as pd

# Caminho para o seu arquivo
caminho_do_arquivo = r'C:\Users\mlzengo\Documents\TJGO\II SEMESTRE\forecast_case\data\raw\20250922092028.csv'

# --- ETAPA 1: LEITURA CORRIGIDA ---
df = pd.read_csv(
    caminho_do_arquivo,
    encoding='utf-8',
    sep=';',
    skiprows=1,
    skipfooter=1,
    header=None,
    engine='python'
)

# --- ETAPA 2: TRANSFORMAR E LIMPAR O DATAFRAME ---

# 2.1. Transpor o DataFrame
df_transposto = df.T

# 2.2. Promover a primeira linha para ser o cabeçalho
df_transposto.columns = df_transposto.iloc[0]
df_transposto = df_transposto[1:]

# 2.3. Limpar e renomear as colunas (COM A CORREÇÃO)
df_final = df_transposto.reset_index(drop=True) # Descarta o índice antigo
df_final.columns = ['Data', 'Valor_IPCA'] # Agora funciona!

# --- ETAPA 3: CONVERTER OS TIPOS DE DADOS ---

# 3.1. Converter a coluna de valor para número (float)
df_final['Valor_IPCA'] = df_final['Valor_IPCA'].str.replace(',', '.').astype(float)

# 3.2. Converter a coluna de data para o formato datetime
meses_map = {
    'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04', 
    'maio': '05', 'junho': '06', 'julho': '07', 'agosto': '08', 
    'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12'
}
df_final[['Mes', 'Ano']] = df_final['Data'].str.split(' ', expand=True)
df_final['Mes_Num'] = df_final['Mes'].str.lower().map(meses_map)
df_final['Data'] = pd.to_datetime('01-' + df_final['Mes_Num'] + '-' + df_final['Ano'], format='%d-%m-%Y')
df_final = df_final[['Data', 'Valor_IPCA']]

# --- VERIFICAÇÃO FINAL ---
print("DataFrame corrigido e formatado:")
display(df_final.head())

print("\nNovas informações do DataFrame:")
df_final.info()

DataFrame corrigido e formatado:


,Data,Valor_IPCA
0,1994-07-01,6.84
1,1994-08-01,1.86
2,1994-09-01,1.53
3,1994-10-01,2.62
4,1994-11-01,2.81



Novas informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Data        374 non-null    datetime64[ns]
 1   Valor_IPCA  374 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 6.0 KB


In [24]:
# Cria uma nova coluna 'Mes' com o número do mês extraído da coluna 'Data'
df_final['Mes'] = df_final['Data'].dt.month

# Exibe o resultado com a nova coluna
display(df_final.head())

,Data,Valor_IPCA,Mes
0,1994-07-01,6.84,7
1,1994-08-01,1.86,8
2,1994-09-01,1.53,9
3,1994-10-01,2.62,10
4,1994-11-01,2.81,11


In [25]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Data        374 non-null    datetime64[ns]
 1   Valor_IPCA  374 non-null    float64       
 2   Mes         374 non-null    int32         
dtypes: datetime64[ns](1), float64(1), int32(1)
memory usage: 7.4 KB


In [26]:


import locale

# Configura o locale para português do Brasil
try:
    locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')
except locale.Error:
    print("Locale 'pt_BR.UTF-8' não suportado. Tentando 'Portuguese_Brazil'.")
    locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil')


# Cria a coluna com o nome do mês
df_final['mes_nome'] = df_final['Data'].dt.strftime('%B').str.capitalize()

# Exibe o resultado
display(df_final.head())

,Data,Valor_IPCA,Mes,mes_nome
0,1994-07-01,6.84,7,Julho
1,1994-08-01,1.86,8,Agosto
2,1994-09-01,1.53,9,Setembro
3,1994-10-01,2.62,10,Outubro
4,1994-11-01,2.81,11,Novembro


In [28]:
import pandas as pd
import os

# --- ETAPA 1: FAZER O RECORTE DA SÉRIE TEMPORAL ---

# Definir as datas de início e fim para o filtro
data_inicio = '2014-01-01'
data_fim = '2024-12-31'

# Aplicar o filtro na coluna 'Data'
# O .copy() é uma boa prática para evitar avisos do Pandas (SettingWithCopyWarning)
df_recorte = df_final[(df_final['Data'] >= data_inicio) & (df_final['Data'] <= data_fim)].copy()


# --- ETAPA 2: SALVAR O NOVO DATAFRAME ---

# Definir o caminho da pasta e o nome do arquivo
caminho_pasta_saida = r'C:\Users\mlzengo\Documents\TJGO\II SEMESTRE\forecast_case\data\processed'
nome_arquivo = 'ipca_2014_2024.csv'
caminho_completo_saida = os.path.join(caminho_pasta_saida, nome_arquivo)

# Criar o diretório de destino, se ele não existir
os.makedirs(caminho_pasta_saida, exist_ok=True)

# Salvar o DataFrame recortado em um novo arquivo CSV
# index=False evita que o índice do DataFrame seja salvo como uma coluna no CSV
df_recorte.to_csv(caminho_completo_saida, index=False)


# --- ETAPA 3: CONFIRMAÇÃO ---

print(f"DataFrame recortado com sucesso!")
print(f"Total de registros no novo DataFrame: {len(df_recorte)}")
print(f"Salvo em: {caminho_completo_saida}")

print("\nPrimeiras linhas do DataFrame recortado:")
display(df_recorte.head())

print("\nÚltimas linhas do DataFrame recortado:")
display(df_recorte.tail())

DataFrame recortado com sucesso!
Total de registros no novo DataFrame: 132
Salvo em: C:\Users\mlzengo\Documents\TJGO\II SEMESTRE\forecast_case\data\processed\ipca_2014_2024.csv

Primeiras linhas do DataFrame recortado:


,Data,Valor_IPCA,Mes,mes_nome
234,2014-01-01,0.55,1,Janeiro
235,2014-02-01,0.69,2,Fevereiro
236,2014-03-01,0.92,3,Marã§o
237,2014-04-01,0.67,4,Abril
238,2014-05-01,0.46,5,Maio



Últimas linhas do DataFrame recortado:


,Data,Valor_IPCA,Mes,mes_nome
361,2024-08-01,-0.02,8,Agosto
362,2024-09-01,0.44,9,Setembro
363,2024-10-01,0.56,10,Outubro
364,2024-11-01,0.39,11,Novembro
365,2024-12-01,0.52,12,Dezembro


## Fonte: "IBGE - Pesquisa Nacional por Amostra de Domicílios Contínua mensal"

In [30]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Taxa de desocupação  3 non-null      object
dtypes: object(1)
memory usage: 156.0+ bytes
